In [ ]:
# install required moduls
!pip install tensorflow tf2onnx

In [30]:
import numpy as np
import pandas as pd
import os
import s3fs
import pyarrow as pa
import pyarrow.parquet as pq
import tensorflow as tf
import tf2onnx
import onnx
import boto3


In [ ]:
bucket = 'mstoffel'
subfolder = 'mstoffel/t15264971/measurements_c8y_Acceleration/'

In [31]:
##
# something along the line ...
##
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(bucket)
parquet_files = [];
for obj in my_bucket.objects.filter(Prefix=subfolder):
     if obj.key.endswith('.parquet'):
        parquet_files.append(obj)
        
parquet_files 

[s3.ObjectSummary(bucket_name='mstoffel', key='mstoffel/t15264971/measurements_c8y_Acceleration/incremental_1687942475/0_2023/0_06/0_28/0_0_0.parquet')]

In [33]:
##
# only use the 1. parquet file
##
parquet_file_1 = parquet_files[0]

s3 = s3fs.S3FileSystem(
    anon=False,
    use_ssl=True,
    client_kwargs={
        "region_name": 'eu-central-1',
        "verify": True,
    }
)



pf = pq.ParquetDataset('s3://' + bucket + '/' + parquet_file_1.key, filesystem=s3)
pf


In [34]:
# show schema
pf.schema

id: string
creationTime: timestamp[ms, tz=UTC]
creationTimeOffset: int32
creationTimeWithOffset: timestamp[ms, tz=UTC]
YEAR: string
MONTH: string
DAY: string
time: timestamp[ms, tz=UTC]
timeOffset: int32
timeWithOffset: timestamp[ms, tz=UTC]
source: string
type: string
c8y_Acceleration.accelerationX.label: string
c8y_Acceleration.accelerationX.value: double
c8y_Acceleration.accelerationY.label: string
c8y_Acceleration.accelerationY.value: double
c8y_Acceleration.accelerationZ.label: string
c8y_Acceleration.accelerationZ.value: double
-- schema metadata --
is.date.correct: 'true'
dremio.arrow.schema.2.1: '{
  "fields" : [ {
    "name" : "id",
    "null' + 2759
dremio.version: '19.11.2-202301280043390504-e5820c5a'

In [15]:
# convert parquet datatset to pandas
data = pf.read().to_pandas()
data.sample(5)

,id,creationTime,creationTimeOffset,creationTimeWithOffset,YEAR,MONTH,DAY,time,timeOffset,timeWithOffset,source,type,c8y_Acceleration.accelerationX.label,c8y_Acceleration.accelerationX.value,c8y_Acceleration.accelerationY.label,c8y_Acceleration.accelerationY.value,c8y_Acceleration.accelerationZ.label,c8y_Acceleration.accelerationZ.value
104,58572466,2023-06-28 08:53:17.418000+00:00,0,2023-06-28 08:53:17.418000+00:00,2023,06,28,2023-04-03 07:36:34+00:00,120,2023-04-03 09:36:34+00:00,5558565188,c8y_Acceleration,moving,0.229004,moving,0.282135,moving,-0.471603
232,58567710,2023-06-28 08:53:12.646000+00:00,0,2023-06-28 08:53:12.646000+00:00,2023,06,28,2023-04-03 07:37:11.500000+00:00,120,2023-04-03 09:37:11.500000+00:00,5558565188,c8y_Acceleration,moving,0.605927,moving,-0.226257,moving,0.917725
160,58574267,2023-06-28 08:53:12.720000+00:00,0,2023-06-28 08:53:12.720000+00:00,2023,06,28,2023-04-03 07:35:39+00:00,120,2023-04-03 09:35:39+00:00,5558565188,c8y_Acceleration,moving,0.464493,moving,0.564468,moving,-0.182236
149,58567179,2023-06-28 08:53:17.529000+00:00,0,2023-06-28 08:53:17.529000+00:00,2023,06,28,2023-04-03 07:24:07+00:00,120,2023-04-03 09:24:07+00:00,5558565188,c8y_Acceleration,sitting,0.007202,sitting,0.023743,sitting,0.998169
256,58574350,2023-06-28 08:53:17.976000+00:00,0,2023-06-28 08:53:17.976000+00:00,2023,06,28,2023-04-03 07:25:14.500000+00:00,120,2023-04-03 09:25:14.500000+00:00,5558565188,c8y_Acceleration,sitting,0.010986,sitting,0.024124,sitting,0.997025


In [16]:
#remove time column from the source
data.pop(data.columns[0])
data.sample(5)

,creationTime,creationTimeOffset,creationTimeWithOffset,YEAR,MONTH,DAY,time,timeOffset,timeWithOffset,source,type,c8y_Acceleration.accelerationX.label,c8y_Acceleration.accelerationX.value,c8y_Acceleration.accelerationY.label,c8y_Acceleration.accelerationY.value,c8y_Acceleration.accelerationZ.label,c8y_Acceleration.accelerationZ.value
582,2023-06-28 08:53:19.048000+00:00,0,2023-06-28 08:53:19.048000+00:00,2023,06,28,2023-04-03 07:35:38.500000+00:00,120,2023-04-03 09:35:38.500000+00:00,5558565188,c8y_Acceleration,moving,0.832062,moving,0.141586,moving,-1.136993
40,2023-06-28 08:53:16.505000+00:00,0,2023-06-28 08:53:16.505000+00:00,2023,06,28,2023-04-03 07:37:06.500000+00:00,120,2023-04-03 09:37:06.500000+00:00,5558565188,c8y_Acceleration,moving,0.994644,moving,-0.007782,moving,-1.044525
598,2023-06-28 08:53:13.264000+00:00,0,2023-06-28 08:53:13.264000+00:00,2023,06,28,2023-04-03 07:25:36+00:00,120,2023-04-03 09:25:36+00:00,5558565188,c8y_Acceleration,sitting,0.011520,sitting,0.024551,sitting,0.997559
391,2023-06-28 08:53:09.276000+00:00,0,2023-06-28 08:53:09.276000+00:00,2023,06,28,2023-04-03 07:35:11+00:00,120,2023-04-03 09:35:11+00:00,5558565188,c8y_Acceleration,moving,-1.053024,moving,0.612946,moving,-0.273682
322,2023-06-28 08:53:09.075000+00:00,0,2023-06-28 08:53:09.075000+00:00,2023,06,28,2023-04-03 07:35:33+00:00,120,2023-04-03 09:35:33+00:00,5558565188,c8y_Acceleration,moving,0.324265,moving,0.331284,moving,-0.842850


In [17]:

# drop unused column and rename colum
data = data.drop( ['creationTime', 'creationTimeOffset','creationTimeWithOffset', 'YEAR', 'MONTH', 'DAY', 'time', 'timeOffset', 'timeWithOffset', 'source', 'type',  'c8y_Acceleration.accelerationX.label', 'c8y_Acceleration.accelerationY.label'  ], axis = 1)

data = data.rename(columns={"c8y_Acceleration.accelerationZ.label": "label" ,"c8y_Acceleration.accelerationX.value": "accelerationX", "c8y_Acceleration.accelerationY.value": "accelerationY","c8y_Acceleration.accelerationZ.value": "accelerationZ"})

data.sample(5)

# pop the label columns
#labels = data.pop("label")
#print(labels)

,accelerationX,accelerationY,label,accelerationZ
280,0.691177,0.321640,moving,0.915924
311,0.494385,0.495193,moving,-0.644180
210,0.661697,0.434265,moving,-0.039047
666,1.057373,-0.079422,moving,-0.920441
474,1.022873,0.290695,moving,0.160690


In [18]:
# pop the label columns
labels = data.pop("label")
print(labels)

0      sitting
1      sitting
2      sitting
3       moving
4      sitting
        ...   
739    sitting
740     moving
741     moving
742     moving
743     moving
Name: label, Length: 744, dtype: object


In [19]:
# normalize data
data = (data - data.mean()) / data.std() 
data.sample(5)

,accelerationX,accelerationY,accelerationZ
580,-0.377754,-0.129798,0.773110
467,-0.170227,-0.515519,0.524468
90,-3.117113,1.546385,-0.831440
637,0.124207,1.264204,-0.047858
634,-0.196646,-0.494570,0.527083


In [20]:
# Reshape data to 3D array
data = data.values.reshape((-1, 3, 1))
print(data)

[[[-0.19453572]
  [-0.49746739]
  [ 0.52405891]]

 [[-0.19928323]
  [-0.49668254]
  [ 0.52419535]]

 [[-0.19844802]
  [-0.50290092]
  [ 0.52412713]]

 ...

 [[ 0.09343603]
  [ 1.13053909]
  [ 0.52278546]]

 [[-1.08680414]
  [ 2.14467738]
  [-1.5577631 ]]

 [[-0.81637182]
  [ 0.41168301]
  [ 0.34215986]]]


In [21]:
# Convert labels to binary
labels = np.where(labels=="moving", 1, 0)

In [22]:
# Split data into train and test sets
split = int(len(data) * 0.8)
x_train = data[:split]
y_train = labels[:split]
x_test = data[split:]
y_test = labels[split:]

In [23]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(3, 1)),
    tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.Reshape((3, 32, 1)),
    tf.keras.layers.MaxPooling2D((2, 1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [24]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [25]:
# Train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30, batch_size=64)

Epoch 1/30
10/10 [==============================] - 1s 27ms/step - loss: 0.6620 - accuracy: 0.6437 - val_loss: 0.6005 - val_accuracy: 0.5369
Epoch 2/30
10/10 [==============================] - 0s 6ms/step - loss: 0.5791 - accuracy: 0.6790 - val_loss: 0.5320 - val_accuracy: 0.5436
Epoch 3/30
10/10 [==============================] - 0s 6ms/step - loss: 0.5213 - accuracy: 0.8050 - val_loss: 0.4790 - val_accuracy: 0.9195
Epoch 4/30
10/10 [==============================] - 0s 8ms/step - loss: 0.4693 - accuracy: 0.9160 - val_loss: 0.4332 - val_accuracy: 0.8859
Epoch 5/30
10/10 [==============================] - 0s 7ms/step - loss: 0.4250 - accuracy: 0.9126 - val_loss: 0.3956 - val_accuracy: 0.8859
Epoch 6/30
10/10 [==============================] - 0s 6ms/step - loss: 0.3898 - accuracy: 0.9059 - val_loss: 0.3632 - val_accuracy: 0.8859
Epoch 7/30
10/10 [==============================] - 0s 7ms/step - loss: 0.3590 - accuracy: 0.9042 - val_loss: 0.3354 - val_accuracy: 0.8859
Epoch 8/30
10/10 [=

In [35]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

5/5 [==============================] - 0s 3ms/step - loss: 0.1039 - accuracy: 0.9732
Test accuracy: 0.9731543660163879


In [36]:
# Use the model to predict label for a single row of data
single_row = np.array([[[0.070770263671875,0.7673492431640625,0.751739501953125]]]).reshape((1, 3, 1))
#single_row = np.array([1.0,1.0,1.0]).reshape((1, 3, 1))
prediction = model.predict(single_row)
prediction 

1/1 [==============================] - 0s 35ms/step


array([[0.99875534]], dtype=float32)

In [37]:
#MODEL_DIR = tempfile.gettempdir()
version = 1
#export_path = os.path.join(MODEL_DIR, str(version))
#print('export_path = {}\n'.format(export_path))

export_path = '../model/cumulocity_classification_model'

# Export the model as a SavedModel
tf.saved_model.save(model, export_path)


print('\nSaved model:' + export_path)
#!ls -l {export_path}

INFO:tensorflow:Assets written to: ../model/cumulocity_classification_model/assets


INFO:tensorflow:Assets written to: ../model/cumulocity_classification_model/assets



Saved model:../model/cumulocity_classification_model


In [38]:
local_model_path = f"{export_path}/model.onnx"
onnx_model, _ = tf2onnx.convert.from_keras(model, opset=13)
onnx.save(onnx_model, local_model_path)

In [39]:
model_sub_folder = 'mstoffel/t15264971/model'
file_name = 'acceleration_model.onnx'

s3_model_path = f"{bucket}/{model_sub_folder}/{file_name}"

s3.put(local_model_path, s3_model_path)

